## Distance Matrix

In [ ]:
#A tool to generate pairwise distances between atoms in a molecule.
#Useful if you would want to calculate the shortest and longest distances in a molecule.
#N.B: SMID REQUIRES an ensemble of 3D conformations that needs to be saved in the /Multiconformer_db folder before running the script

#### Please attribute back to Amin Aghamohammadi . Thank you. 

In [ ]:
!pip install ase numpy

In [ ]:
import sys
import os
import glob
import numpy as np
import ase.io
import io

In [ ]:
# Confirm the current working directory by running this cell.
Notebook_location= os.getcwd()
Notebook_location

In [ ]:
# Walk through the Multiconformer_db folder to find the SDF file.
folderwalk = os.walk('Multiconformer_db')
O1F = next(folderwalk)[2][0]
O1F

In [ ]:
# Grabs the filname for the SDF.
O2F= os.path.splitext(O1F)[0]
print(O2F)

In [ ]:
# A function to split the multiconformer SDF into separate SD files, placing them in the runtime_folder.
def split_sdf(input_file, max_structures_per_file=1):    
    with open(input_file, 'r') as f:
        sdf_data = f.read().strip().split("$$$$\n")
    num_output_files = (len(sdf_data) // max_structures_per_file) + 1
    sdf_chunks = [sdf_data[i:i+max_structures_per_file] for i in range(0, len(sdf_data), max_structures_per_file)]
    for i, chunk in enumerate(sdf_chunks):
        output_file = f"runtime_folder/{O2F}_{i + 1}.sdf"
        with open(output_file, 'w') as f_out:
            f_out.write("".join(chunk))

In [ ]:
if __name__ == "__main__":
    input_sdf_file = "/Multiconformer_db/"+f"{O2F}.sdf"  
    max_structures_per_file = 1  
    split_sdf(input_sdf_file, max_structures_per_file)

In [ ]:
# Iterates through each SD file and calculates all the distances.
# .max() function grabs the maximum distance.
# Saves a txt file with the maximum distance.
directory = '/runtime_folder'
for filename in glob.iglob(f'{directory}/*'):
    conf=ase.io.read(filename)
    dm=conf.get_all_distances()
    maxdm=dm.max()
    with open('runtime_folder/calc.txt', 'a') as f:
        print(maxdm, file=f)

In [ ]:
with open('runtime_folder/calc.txt') as file:
    max_imd = [line.rstrip() for line in file]

In [ ]:
smid=min(max_imd)
print(smid)

In [ ]:
# Caution! Running this will REMOVE all the files in the runtime_folder! 
contents = glob.glob('runtime_folder/*')
for f in contents:
    os.remove(f)

In [ ]:
# Writes out a final txt file with the name of SDF followed by SMID. 
with open('calculated.txt', 'a') as f:
    print(O2F,',',smid, file=f)